In [ ]:
import requests
from langchain.retrievers import WikipediaRetriever
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from pydantic import BaseModel, Field
from typing import Type
from bs4 import BeautifulSoup
from langchain.schema import SystemMessage

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)

# Define Argument Schema for Tools
class SearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class URLArgsSchema(BaseModel):
    url: str = Field(description="The URL of the webpage to scrape")

class SaveFileArgsSchema(BaseModel):
    content: str = Field(description="The research content to save into a .txt file")

# 📌 Wikipedia Search Tool
class WikiSearchTool(BaseTool):
    name: str = "WikiSearchTool"
    description: str = "Search Wikipedia for relevant information and extract the text from the page."
    
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query: str):
        retriever = WikipediaRetriever()
        result = retriever.get_relevant_documents(query)
        return "\n\n".join([doc.page_content for doc in result]) if result else "No relevant Wikipedia pages found."

# 📌 DuckDuckGo Search Tool
class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGoSearchTool"
    description: str = "Use DuckDuckGo to search the web for relevant links."
    
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        result = ddg.run(query)
        return result

# 📌 Web Scraping Tool
class WebScraperTool(BaseTool):
    name: str = "WebScraperTool"
    description: str = "Extract the text content of a given webpage."

    args_schema: Type[URLArgsSchema] = URLArgsSchema

    def _run(self, url: str):
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text(separator="\n").strip()
            return text[:2000]  # Limit output to 2000 characters
        except Exception as e:
            return f"Error scraping website: {str(e)}"

# 📌 Save Research to File Tool
class SaveResearchTool(BaseTool):
    name: str = "SaveResearchTool"
    description: str = "Save the research content into a .txt file."

    args_schema: Type[SaveFileArgsSchema] = SaveFileArgsSchema

    def _run(self, content: str):
        try:
            with open("research.txt", "w", encoding="utf-8") as f:
                f.write(content)
            return "Research saved to research.txt"
        except Exception as e:
            return f"Error saving file: {str(e)}"

# 📌 Initialize Agent
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        DuckDuckGoSearchTool(),
        WikiSearchTool(),
        WebScraperTool(),
        SaveResearchTool()
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are an AI research assistant that gathers information using multiple sources.

            1. Start by choosing either DuckDuckGoSearchTool or WikiSearchTool randomly.
            2. Alternate between tools to ensure variety (e.g., if you used DuckDuckGo last, use Wikipedia next).
            3. If DuckDuckGoSearchTool returns a URL, use WebScraperTool to extract the page content.
            4. Collect and summarize information from at least two sources.
            5. Once you have summarized the research, you MUST use SaveResearchTool to save the final summary to a .txt file.
            6. Do NOT skip the SaveResearchTool step — this is how your task is finalized.

            Your final step must always be to call SaveResearchTool with your research summary.
            """
        )
    }
)

# 📌 Run the agent with the required query
query = "Research about the XZ backdoor"
result = agent.invoke(query)

print(result)

In [2]:

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 30 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [3]:
import openai as client

assistant = client.beta.assistants.create(
    name = "Investor Assistant",
    instructions = "You help users do research on publicly traded cmpanies and you help users decide if they should buy the storck or not",
    model ="gpt-4o-mini",
    tools=functions
)